# Study case: the 2nd classified solution of the challenge

Ideally in this notebook I will do 3 things:
1) study one of the top solutions of the challenge
2) replicate the feature engineering 
3) implement a version of the solution, as well as other models such as a DNN

## 2nd classified solution

In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBClassifier
from xgboost import plot_importance

A few global variables are set. One is the path to the training and test data, the other is a state to remove randomness of some algorithms (for reproduction purposes). In the same cell, we define the features and the target variables.

In [5]:
data_dir = '../Datasets/killer-shrimp-invasion/'
RANDOM_STATE = 0

train = pd.read_csv(data_dir + 'train.csv')
test = pd.read_csv(data_dir + 'test.csv')

# split data
Y_train = train['Presence']
ID_train = train['pointid']
X_train = train.drop(['Presence', 'pointid'], axis=1)
ID_test = test['pointid']
X_test = test.drop(['pointid'], axis=1)